In [2]:
import os
import SimpleITK as sitk
import pydicom
import numpy as np
import matplotlib.pyplot as plt

def load_dicom_images_from_folder(folder_path):
    dicom_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]
    dicom_files.sort(key=lambda f: int(''.join(filter(str.isdigit, os.path.basename(f)))))
    return dicom_files

def read_dicom_series(dicom_files):
    slices = [pydicom.dcmread(f) for f in dicom_files]
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    return slices

def convert_to_volume(dicom_files):
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(dicom_files)
    image = reader.Execute()
    return image

def load_volume(folder_path):
    dicom_files = load_dicom_images_from_folder(folder_path)
    slices = read_dicom_series(dicom_files)
    volume = convert_to_volume(dicom_files)
    return sitk.GetArrayFromImage(volume)

def get_tumor_folders(main_directory):
    tumor_folders = [os.path.join(main_directory, folder) for folder in os.listdir(main_directory)]
    return tumor_folders

def sum_tumor_volumes(tumor_volumes):
    combined_array = sitk.GetArrayFromImage(tumor_volumes[0])
    for volume in tumor_volumes[1:]:
        volume_array = sitk.GetArrayFromImage(volume)
        combined_array += volume_array
        combined_volume = sitk.GetImageFromArray(combined_array)
        combined_volume.CopyInformation(tumor_volumes[0])

    return combined_volume

def load_and_sum_tumors(main_directory):
    tumor_folders = get_tumor_folders(main_directory)
    tumor_volumes = []

    for folder in tumor_folders:
        dicom_files = load_dicom_images_from_folder(folder)
        tumor_volume = convert_to_volume(dicom_files)
        tumor_volumes.append(tumor_volume)

    combined_volume = sum_tumor_volumes(tumor_volumes)
    return sitk.GetArrayFromImage(combined_volume)




ct_volume = load_volume(r"D:\CodePro\VsCodePythonPro\MachinelearningClass\EX7\3Dircadb1.1\PATIENT_DICOM")
liver_mask_volume = load_volume(r"D:\CodePro\VsCodePythonPro\MachinelearningClass\EX7\3Dircadb1.1\MASKS_DICOM\liver")
tumors_mask_volume = load_and_sum_tumors(r"D:\CodePro\VsCodePythonPro\MachinelearningClass\EX7\3Dircadb1.1\MASKS_DICOM\tumors_mask_volume")

print(f"Combined volume size: {tumors_mask_volume.shape}")

Combined volume size: (129, 512, 512)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,14))

plt.subplot(4, 3, 1)
plt.imshow(ct_volume[60, :, :], cmap='gray')
plt.title("CT Image - Slice 60")
plt.axis("off")

plt.subplot(4, 3, 2)
plt.imshow(liver_mask_volume[60, :, :], cmap='gray')
plt.title("Liver Mask - Slice 60")
plt.axis("off")

plt.subplot(4, 3, 3)
plt.imshow(tumors_mask_volume[60, :, :], cmap='gray')
plt.title("Tumor Mask - Slice 60")
plt.axis("off")

plt.subplot(4, 3, 4)
plt.imshow(ct_volume[80, :, :], cmap='gray')
plt.title("CT Image - Slice 80")
plt.axis("off")

plt.subplot(4, 3, 5)
plt.imshow(liver_mask_volume[80, :, :], cmap='gray')
plt.title("Liver Mask - Slice 80")
plt.axis("off")

plt.subplot(4, 3, 6)
plt.imshow(tumors_mask_volume[80, :, :], cmap='gray')
plt.title("Tumor Mask - Slice 80")
plt.axis("off")

plt.subplot(4, 3, 7)
plt.imshow(ct_volume[111, :, :], cmap='gray')
plt.title("CT Image - Slice 111")
plt.axis("off")

plt.subplot(4, 3, 8)
plt.imshow(liver_mask_volume[111, :, :], cmap='gray')
plt.title("Liver Mask - Slice 111")
plt.axis("off")

plt.subplot(4, 3, 9)
plt.imshow(tumors_mask_volume[111, :, :], cmap='gray')
plt.title("Tumor Mask - Slice 111")
plt.axis("off")

plt.subplot(4, 3, 10)
plt.imshow(ct_volume[117, :, :], cmap='gray')
plt.title("CT Image - Slice 117")
plt.axis("off")

plt.subplot(4, 3, 11)
plt.imshow(liver_mask_volume[117, :, :], cmap='gray')
plt.title("Liver Mask - Slice 117")
plt.axis("off")

plt.subplot(4, 3, 12)
plt.imshow(tumors_mask_volume[117, :, :], cmap='gray')
plt.title("Tumor Mask - Slice 117")
plt.axis("off")

plt.tight_layout()
plt.show()

KMeans


In [4]:
ct_volume = np.clip(ct_volume, 0, 255)
liver_mask_volume = liver_mask_volume / 255
tumors_mask_volume = tumors_mask_volume / 255

In [ ]:
slices = [60, 80, 111, 117]

fig, axes = plt.subplots(1, len(slices), figsize=(15, 5))

for idx, slice_idx in enumerate(slices):
    ct = ct_volume[slice_idx, :, :]
    liver_mask = liver_mask_volume[slice_idx, :, :]

    liver_extracted_ct = ct * liver_mask

    axes[idx].imshow(liver_extracted_ct, cmap='gray')
    axes[idx].set_title(f"Slice {slice_idx}")
    axes[idx].axis("off")

plt.suptitle("Liver Extracted CT for Selected Slices", fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.cluster import KMeans

slices = [60, 80, 111, 117]

for slice_idx in slices:
    ct = ct_volume[slice_idx, :, :]
    liver_mask = liver_mask_volume[slice_idx, :, :]
    
    liver_extracted_ct = ct * liver_mask

    liver_pixels = liver_extracted_ct[liver_mask > 0]
    liver_pixels = liver_pixels.reshape(-1, 1)  # Reshape for scaling

    scaler = StandardScaler()
    liver_pixels_scaled = scaler.fit_transform(liver_pixels)

    k = 2
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(liver_pixels_scaled)

    labels = kmeans.predict(liver_pixels_scaled)

    clustered_image = np.zeros(liver_extracted_ct.shape, dtype=int)
    clustered_image[liver_mask > 0] = labels  # Assign labels to the pixels based on mask

    unique_clusters = np.unique(labels)
    num_clusters = len(unique_clusters)

    plt.figure(figsize=(8, 3))
    for i in range(num_clusters):
        plt.subplot(1, num_clusters, i + 1)
        mask = clustered_image == i 
        plt.imshow(mask, cmap='gray')
        plt.title(f'Cluster {i}')
        plt.axis('off')

    plt.suptitle(f'KMeans Clusters for Slice {slice_idx}')
    plt.tight_layout()
    plt.show()



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.utils import shuffle
from skimage.morphology import erosion, dilation, disk
from sklearn.metrics import jaccard_score, f1_score

slices = [60, 80, 111, 117]

for slice_index in slices:
    ct_slice = ct_volume[slice_index, :, :]
    liver_mask_slice = liver_mask_volume[slice_index, :, :]
    tumor_mask_slice = tumors_mask_volume[slice_index, :, :]
    
    liver_extracted_ct_slice = ct_slice * liver_mask_slice

    liver_pixels_slice = liver_extracted_ct_slice[liver_mask_slice > 0]
    liver_pixels_slice = liver_pixels_slice.reshape(-1, 1)

    scaler = StandardScaler()
    liver_pixels_scaled_slice = scaler.fit_transform(liver_pixels_slice)

    k = 2
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(liver_pixels_scaled_slice)
    labels = kmeans.predict(liver_pixels_scaled_slice)

    clustered_image = np.zeros(liver_extracted_ct_slice.shape, dtype=int)
    clustered_image[liver_mask_slice > 0] = labels
    unique_clusters = np.unique(labels)
    num_clusters = len(unique_clusters)

    plt.figure(figsize=(6, 3))
    for i in range(num_clusters):
        plt.subplot(1, num_clusters, i + 1)
        mask = clustered_image == i
        plt.imshow(mask, cmap='gray')
        plt.title(f'Cluster {i}')
        plt.axis('off')

    plt.suptitle(f'KMeans Clusters for Slice {slice_index}', fontsize=8)
    plt.tight_layout()
    plt.show()

    sample_size = 50000
    n_pixels = liver_pixels_scaled_slice.shape[0]
    indices = shuffle(np.arange(n_pixels), random_state=42)[:sample_size]
    pixels_sample = liver_pixels_scaled_slice[indices]

    silhouette_scores = []
    best_score = -1
    best_k = None
    best_kmeans = None

    print(f"Evaluating silhouette scores for Slice {slice_index}:")
    for k in range(2, 10):
        kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
        kmeans.fit(pixels_sample)
        sample_labels = kmeans.predict(pixels_sample)
        score = silhouette_score(pixels_sample, sample_labels)
        silhouette_scores.append(score)
        print(f"k: {k:<2} , Silhouette Score: {score:.3f}")

        if score > best_score:
            best_score = score
            best_k = k
            best_kmeans = kmeans

    labels = best_kmeans.labels_
    clustered_image = np.zeros(liver_extracted_ct_slice.shape, dtype=int)
    clustered_image[liver_mask_slice > 0] = labels

    unique_clusters = np.unique(labels)
    num_clusters = len(unique_clusters)

    plt.figure(figsize=(6, 3))
    for i in range(num_clusters):
        plt.subplot(1, num_clusters, i + 1)
        mask = clustered_image == i
        plt.imshow(mask, cmap='gray')
        plt.title(f'Cluster {i}')
        plt.axis('off')

    plt.figure(figsize=(6, 3))
    plt.plot(np.arange(2, 10), silhouette_scores, 'bo-')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title(f'Silhouette Score vs Number of Clusters for Slice {slice_index}')
    plt.grid()
    plt.show()

    print(f"Best number of Clusters (K) for Slice {slice_index} based on silhouette score: {best_k}")
    print(f"Best silhouette score for Slice {slice_index}: {best_score:.3f}")
    labels = best_kmeans.predict(liver_pixels_scaled_slice)

    clustered_image = np.zeros(liver_extracted_ct_slice.shape, dtype=int)
    clustered_image[liver_mask_slice > 0] = labels

    pixel_counts = np.bincount(labels)
    min_cluster_id = np.argmin(pixel_counts)
    min_cluster_count = pixel_counts[min_cluster_id]

    print(f"Cluster {min_cluster_id} for Slice {slice_index} has the minimum number of pixels: {min_cluster_count}")
    min_cluster_mask = (clustered_image == min_cluster_id)

    structuring_element = disk(3)
    eroded_mask = erosion(min_cluster_mask, structuring_element)
    structuring_element = disk(10)
    dilated_mask = dilation(eroded_mask, structuring_element)

    fig, axes = plt.subplots(1, 3, figsize=(9, 3))
    axes[0].imshow(min_cluster_mask, cmap='gray')
    axes[0].set_title(f'Original Cluster {min_cluster_id} Mask')
    axes[0].axis('off')
    axes[1].imshow(eroded_mask, cmap='gray')
    axes[1].set_title(f'Eroded Mask (Cluster {min_cluster_id})')
    axes[1].axis('off')
    axes[2].imshow(dilated_mask, cmap='gray')
    axes[2].set_title(f'Dilated Mask (Cluster {min_cluster_id})')
    axes[2].axis('off')
    plt.suptitle(f'Morphological Operations on Cluster Mask for Slice {slice_index}', fontsize=8)
    plt.tight_layout()
    plt.show()

    predicted_flat = dilated_mask.flatten()
    ground_truth_flat = tumor_mask_slice.flatten()

    iou = jaccard_score(ground_truth_flat, predicted_flat)
    dice = f1_score(ground_truth_flat, predicted_flat)

    print(f"\nEvaluation Metrics for Tumor Segmentation for Slice {slice_index}:")
    print(f"Intersection over Union (IoU): {iou:.4f}")
    print(f"Dice Coefficient: {dice:.4f}")

    ct_rgb = np.stack([liver_extracted_ct_slice] * 3, axis=-1)
    predicted_overlay = np.zeros_like(ct_rgb)
    ground_truth_overlay = np.zeros_like(ct_rgb)

    predicted_overlay[dilated_mask == 1] = [255, 0, 0]
    ground_truth_overlay[tumor_mask_slice == 1] = [0, 255, 0]

    combined_overlay = ct_rgb.copy()
    combined_overlay = np.where(predicted_overlay.any(axis=-1, keepdims=True), predicted_overlay, combined_overlay)
    combined_overlay = np.where(ground_truth_overlay.any(axis=-1, keepdims=True), ground_truth_overlay, combined_overlay)

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(liver_extracted_ct_slice, cmap='gray')
    plt.title(f'Original ct Image for Slice {slice_index}')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(liver_extracted_ct_slice, cmap='gray')
    plt.imshow(predicted_overlay, alpha=0.3)
    plt.title(f'Predicted Tumor Mask (Red) for Slice {slice_index}')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(liver_extracted_ct_slice, cmap='gray')
    plt.imshow(ground_truth_overlay, alpha=0.3)
    plt.title(f'Ground Truth Tumor Mask (Green) for Slice {slice_index}')
    plt.axis('off')

    plt.suptitle(f'Tumor Segmentation Results for Slice {slice_index}', fontsize=8)
    plt.tight_layout()
    plt.show()

    predicted_tumor_mask = np.logical_and(dilated_mask, liver_mask_slice)
    predicted_overlay = np.zeros_like(ct_rgb)
    predicted_overlay[predicted_tumor_mask == 1] = [255, 0, 0]
    combined_overlay = np.where(predicted_overlay.any(axis=-1, keepdims=True), predicted_overlay, combined_overlay)

    predicted_flat = predicted_tumor_mask.flatten()
    ground_truth_flat = tumor_mask_slice.flatten()

    iou = jaccard_score(ground_truth_flat, predicted_flat)
    dice = f1_score(ground_truth_flat, predicted_flat)

    print(f"\nEvaluation Metrics for Tumor Segmentation for Slice {slice_index}:")
    print(f"Intersection over Union (IoU): {iou:.4f}")
    print(f"Dice Coefficient: {dice:.4f}")

    plt.figure(figsize=(3, 3))
    plt.imshow(liver_extracted_ct_slice, cmap='gray')
    plt.imshow(predicted_overlay, alpha=0.3)
    plt.imshow(ground_truth_overlay, alpha=0.3)
    plt.title(f'Predicted (Red) vs Ground Truth (Green) Tumor Masks for Slice {slice_index}')
    plt.axis('off')
    plt.show()

    print("\n_________________________________________next_________________________________________\n")

MiniBatchKMeans



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from skimage.morphology import erosion, dilation, disk
from sklearn.metrics import jaccard_score, f1_score

slices = [60, 80, 111, 117]

for slice_index in slices:
    ct_slice = ct_volume[slice_index, :, :]
    liver_mask_slice = liver_mask_volume[slice_index, :, :]
    tumor_mask_slice = tumors_mask_volume[slice_index, :, :]

    liver_extracted_ct_slice = ct_slice * liver_mask_slice

    liver_pixels_slice = liver_extracted_ct_slice[liver_mask_slice > 0]
    liver_pixels_slice = liver_pixels_slice.reshape(-1, 1)

    scaler = StandardScaler()
    liver_pixels_scaled_slice = scaler.fit_transform(liver_pixels_slice)

    sample_size = 50000
    n_pixels = liver_pixels_scaled_slice.shape[0]
    indices = shuffle(np.arange(n_pixels), random_state=42)[:sample_size]
    pixels_sample = liver_pixels_scaled_slice[indices]

    silhouette_scores = []
    best_score = -1
    best_k = None
    best_kmeans = None

    print(f"Evaluating and plotting clusters and silhouette diagrams for Slice {slice_index}:")

    for k in range(2, 10):
        minibatch_kmeans = MiniBatchKMeans(n_clusters=k, batch_size=100, random_state=42)
        minibatch_kmeans.fit(pixels_sample)

        sample_labels = minibatch_kmeans.predict(pixels_sample)
        score = silhouette_score(pixels_sample, sample_labels)
        silhouette_scores.append(score)

        print(f"{k:<2}, Silhouette Score: {score:.3f}")

        if score > best_score:
            best_score = score
            best_k = k
            best_kmeans = minibatch_kmeans

    labels = best_kmeans.predict(liver_pixels_scaled_slice)

    clustered_image = np.zeros(liver_extracted_ct_slice.shape, dtype=int)
    clustered_image[liver_mask_slice > 0] = labels

    unique_clusters = np.unique(labels)
    num_clusters = len(unique_clusters)

    plt.figure(figsize=(6, 3))
    for i in range(num_clusters):
        plt.subplot(1, num_clusters, i + 1)
        mask = clustered_image == i
        plt.imshow(mask, cmap='gray')
        plt.title(f'Cluster {i}')
        plt.axis('off')

    plt.figure(figsize=(6, 3))
    plt.plot(np.arange(2, 10), silhouette_scores, 'bo-')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title(f'Silhouette Score vs Number of Clusters for Slice {slice_index}')
    plt.grid()
    plt.show()

    print(f"Best number of Clusters (K) for Slice {slice_index} based on silhouette score: {best_k}")
    print(f"Best silhouette score for Slice {slice_index}: {best_score:.3f}")

    labels = best_kmeans.predict(liver_pixels_scaled_slice)
    clustered_image = np.zeros(liver_extracted_ct_slice.shape, dtype=int)
    clustered_image[liver_mask_slice > 0] = labels

    pixel_counts = np.bincount(labels)
    min_cluster_id = np.argmin(pixel_counts)
    min_cluster_count = pixel_counts[min_cluster_id]

    print(f"Cluster {min_cluster_id} for Slice {slice_index} has the minimum number of pixels: {min_cluster_count}")
    min_cluster_mask = (clustered_image == min_cluster_id)

    structuring_element = disk(3)
    eroded_mask = erosion(min_cluster_mask, structuring_element)
    structuring_element = disk(10)
    dilated_mask = dilation(eroded_mask, structuring_element)

    fig, axes = plt.subplots(1, 3, figsize=(9, 3))
    axes[0].imshow(min_cluster_mask, cmap='gray')
    axes[0].set_title(f'Original Cluster {min_cluster_id} Mask')
    axes[0].axis('off')
    axes[1].imshow(eroded_mask, cmap='gray')
    axes[1].set_title(f'Eroded Mask (Cluster {min_cluster_id})')
    axes[1].axis('off')
    axes[2].imshow(dilated_mask, cmap='gray')
    axes[2].set_title(f'Dilated Mask (Cluster {min_cluster_id})')
    axes[2].axis('off')
    plt.suptitle(f'Morphological Operations on Cluster Mask for Slice {slice_index}', fontsize=16)
    plt.tight_layout()
    plt.show()

    predicted_flat = dilated_mask.flatten()
    ground_truth_flat = tumor_mask_slice.flatten()

    iou = jaccard_score(ground_truth_flat, predicted_flat)
    dice = f1_score(ground_truth_flat, predicted_flat)

    print(f"\nEvaluation Metrics for Tumor Segmentation for Slice {slice_index}:")
    print(f"Intersection over Union (IoU): {iou:.4f}")
    print(f"Dice Coefficient: {dice:.4f}")

    ct_rgb = np.stack([liver_extracted_ct_slice] * 3, axis=-1)
    predicted_overlay = np.zeros_like(ct_rgb)
    ground_truth_overlay = np.zeros_like(ct_rgb)

    predicted_overlay[dilated_mask == 1] = [255, 0, 0]
    ground_truth_overlay[tumor_mask_slice == 1] = [0, 255, 0]

    combined_overlay = ct_rgb.copy()
    combined_overlay = np.where(predicted_overlay.any(axis=-1, keepdims=True), predicted_overlay, combined_overlay)
    combined_overlay = np.where(ground_truth_overlay.any(axis=-1, keepdims=True), ground_truth_overlay, combined_overlay)

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(liver_extracted_ct_slice, cmap='gray')
    plt.title(f'Original CT Image for Slice {slice_index}')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(liver_extracted_ct_slice, cmap='gray')
    plt.imshow(predicted_overlay, alpha=0.3)
    plt.title(f'Predicted Tumor Mask (Red) for Slice {slice_index}')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(liver_extracted_ct_slice, cmap='gray')
    plt.imshow(ground_truth_overlay, alpha=0.3)
    plt.title(f'Ground Truth Tumor Mask (Green) for Slice {slice_index}')
    plt.axis('off')

    plt.suptitle(f'Tumor Segmentation Results for Slice {slice_index}', fontsize=16)
    plt.tight_layout()
    plt.show()

    predicted_tumor_mask = np.logical_and(dilated_mask, liver_mask_slice)
    predicted_overlay = np.zeros_like(ct_rgb)
    predicted_overlay[predicted_tumor_mask == 1] = [255, 0, 0]
    combined_overlay = np.where(predicted_overlay.any(axis=-1, keepdims=True), predicted_overlay, combined_overlay)

    predicted_flat = predicted_tumor_mask.flatten()
    ground_truth_flat = tumor_mask_slice.flatten()

    iou = jaccard_score(ground_truth_flat, predicted_flat)
    dice = f1_score(ground_truth_flat, predicted_flat)

    print(f"\nEvaluation Metrics for Tumor Segmentation for Slice {slice_index}:")
    print(f"Intersection over Union (IoU): {iou:.4f}")
    print(f"Dice Coefficient: {dice:.4f}")

    plt.figure(figsize=(3, 3))
    plt.imshow(liver_extracted_ct_slice, cmap='gray')
    plt.imshow(predicted_overlay, alpha=0.3)
    plt.imshow(ground_truth_overlay, alpha=0.3)
    plt.title(f'Predicted (Red) vs Ground Truth (Green) Tumor Masks for Slice {slice_index}')
    plt.axis('off')
    plt.show()

    print("\n_________________________________________next_________________________________________\n")

DBScan with Grid Search


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import jaccard_score, f1_score
from skimage.morphology import erosion, dilation, disk
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler



best_iou,best_dice = -1, -1
best_min_samples = None

min_samples_range = range(400, 1401, 100)


ct = ct_volume[80, :, :]
liver_mask = liver_mask_volume[80, :, :]
tumor_mask = tumors_mask_volume[80, :, :]
liver_extracted_ct = ct * liver_mask

liver_pixels = liver_extracted_ct[liver_mask > 0]
liver_pixels = liver_pixels.reshape(-1, 1)

scaler = StandardScaler()
liver_pixels_scaled = scaler.fit_transform(liver_pixels)




for min_samples in min_samples_range:
    
    dbscan = DBSCAN(eps=0.07, min_samples=min_samples, metric="l2")
    labels = dbscan.fit_predict(liver_pixels_scaled)

    db_cluster_image = np.zeros(liver_extracted_ct.shape, dtype=int)
    db_cluster_image[liver_mask > 0] = labels

    selected_cluster_mask = (db_cluster_image == -1)

    structuring_element = disk(3)
    eroded_mask = erosion(selected_cluster_mask, structuring_element)
    structuring_element = disk(10)
    dilated_mask = dilation(eroded_mask, structuring_element)

    predicted_tumor_mask = np.logical_and(dilated_mask, liver_mask)

    predicted_flat = predicted_tumor_mask.flatten()
    ground_truth_flat = tumor_mask.flatten()

    iou = jaccard_score(ground_truth_flat, predicted_flat)
    dice = f1_score(ground_truth_flat, predicted_flat)

    print(f"min_samples={min_samples}, IoU={iou:.4f}, Dice={dice:.4f}")

    if iou > best_iou or (iou == best_iou and dice > best_dice):
        best_iou = iou
        best_dice = dice
        best_min_samples = min_samples

print("\n__________________________________________________________________________________\n")
print("Best Parameters:")
print(f"min_samples={best_min_samples}, Best IoU={best_iou:.4f}, Best Dice={best_dice:.4f}")




In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.07, min_samples=900, metric="l2")
labels = dbscan.fit_predict(liver_pixels_scaled)

db_cluster_image = np.zeros(liver_extracted_ct.shape, dtype=int)
db_cluster_image[liver_mask > 0] = labels

unique_clusters = np.unique(labels)
num_clusters = len(unique_clusters)

plt.figure(figsize=(8, 4))
for i in range(num_clusters):
    mask = db_cluster_image == unique_clusters[i]
    plt.subplot(1, num_clusters, i + 1)
    plt.imshow(mask, cmap='gray')
    plt.title(f'Cluster {unique_clusters[i]}')
    plt.axis('off')

plt.suptitle('DBSCAN Clusters', fontsize=16)
plt.tight_layout()
plt.show()

Optimal GMM Components



In [ ]:
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import numpy as np

component_range = range(1, 11)

aic_scores = []
bic_scores = []

ct = ct_volume[117, :, :]
liver_mask = liver_mask_volume[117, :, :]
tumor_mask = tumors_mask_volume[117, :, :]
liver_extracted_ct117 = ct * liver_mask

liver_pixels = liver_extracted_ct117[liver_mask > 0]
liver_pixels = liver_pixels.reshape(-1, 1)

scaler = StandardScaler()
liver_pixels_scaled = scaler.fit_transform(liver_pixels)


for n_components in component_range:
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(liver_pixels_scaled)  # Fit GMM to the data
    aic_scores.append(gmm.aic(liver_pixels_scaled))  # Append AIC
    bic_scores.append(gmm.bic(liver_pixels_scaled))  # Append BIC

plt.figure(figsize=(6, 4))
plt.plot(component_range, aic_scores, label='AIC', marker='o')
plt.plot(component_range, bic_scores, label='BIC', marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Score')
plt.title('AIC and BIC Scores for GMM')
plt.legend()
plt.grid()
plt.show()

best_n_components_aic = component_range[np.argmin(aic_scores)]
best_n_components_bic = component_range[np.argmin(bic_scores)]

print(f"Best number of components based on AIC: {best_n_components_aic}")
print(f"Best number of components based on BIC: {best_n_components_bic}")

gmm = GaussianMixture(n_components=best_n_components_bic, random_state=42)
gmm.fit(liver_pixels_scaled)

labels = gmm.predict(liver_pixels_scaled)

gmm_cluster_image = np.zeros(liver_extracted_ct117.shape, dtype=int)
gmm_cluster_image[liver_mask > 0] = labels

unique_clusters = np.unique(labels)
num_clusters = len(unique_clusters)

plt.figure(figsize=(6, 3))
for i in range(num_clusters):
    plt.subplot(1, num_clusters, i + 1)
    mask = gmm_cluster_image == unique_clusters[i]
    plt.imshow(mask, cmap='gray')
    plt.title(f'Cluster {unique_clusters[i]}')
    plt.axis('off')

plt.suptitle('Optimal GMM Clusters', fontsize=16)
plt.tight_layout()
plt.show()

selected_cluster_mask = (gmm_cluster_image == 1)

plt.figure(figsize=(3, 7))
plt.imshow(selected_cluster_mask, cmap="gray")
plt.title(f'Cluster 1 Mask')
plt.axis('off')
plt.show()

from skimage.morphology import square, disk, erosion, dilation

structuring_element = square(4)
eroded_mask = erosion(selected_cluster_mask, structuring_element)

plt.figure(figsize=(3,7))
plt.imshow(eroded_mask, cmap='gray')
plt.title(f'Cluster Eroded Mask')
plt.axis('off')
plt.show()

structuring_element = disk(14)
dilated_mask = dilation(eroded_mask, structuring_element)

plt.figure(figsize=(3, 7))
plt.imshow(dilated_mask, cmap='gray')
plt.title(f'Cluster Dilated Mask')
plt.axis('off')
plt.show()

ct_rgb = np.stack([liver_extracted_ct117] * 3, axis=-1)

predicted_overlay = np.zeros_like(ct_rgb)
ground_truth_overlay = np.zeros_like(ct_rgb)

predicted_overlay[dilated_mask == 1] = [255, 0, 0] 
ground_truth_overlay[tumor_mask == 1] = [0, 255, 0]  

combined_overlay = ct_rgb.copy()
combined_overlay = np.where(predicted_overlay.any(axis=-1, keepdims=True), predicted_overlay, combined_overlay)
combined_overlay = np.where(ground_truth_overlay.any(axis=-1, keepdims=True), ground_truth_overlay, combined_overlay)

plt.figure(figsize=(8, 3))
plt.subplot(1, 3, 1)
plt.imshow(liver_extracted_ct117, cmap='gray')
plt.title('Original CT Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(liver_extracted_ct117, cmap='gray')
plt.imshow(predicted_overlay, alpha=0.5) 
plt.title('Predicted Tumor Mask (Red)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(liver_extracted_ct117, cmap='gray')
plt.imshow(ground_truth_overlay, alpha=0.5)  
plt.title('Ground Truth Tumor Mask (Green)')
plt.axis('off')

plt.suptitle('Tumor Segmentation Results', fontsize=16)
plt.tight_layout()
plt.show()

plt.figure(figsize=(3,3))
plt.imshow(liver_extracted_ct117, cmap='gray')
plt.imshow(predicted_overlay, alpha=0.3)  
plt.imshow(ground_truth_overlay, alpha=0.3)  
plt.title('Predicted (Red) vs Ground Truth (Green) Tumor Masks')
plt.axis('off')
plt.show()
